In [1]:
import pandas as pd
import numpy as np
import os
import json
import sys


PATH_ROOT  = os.path.dirname(os.path.abspath(os.getcwd()))

if PATH_ROOT not in sys.path:
    sys.path.append(PATH_ROOT)
    print("ROOT JUST ADDED")



ROOT JUST ADDED


In [2]:
print(PATH_ROOT)

/Users/amir/Desktop/GitHub/ClinicalAI-Toolkit


In [3]:
def read_data(file_path: str) -> pd.DataFrame:
    """
    Reads data from a CSV or JSON file.

    Args:
        file_path (str): Path to the data file.

    Returns:
        pd.DataFrame: A DataFrame containing the data.
    """

    if not os.path.exists(file_path):
        raise FileExistsError(f"The file {file_path} does not exist")
    if file_path.endswith(".csv"):
        return pd.read_csv(file_path)
    elif file_path.endswith(".json"):
        with open(file_path,'r') as f:
            data = json.load(f)
        return pd.DataFrame(data)
    else:
        raise ValueError("Unsupported file format. Please provide a csv or json file.")

    

In [17]:
from config import RAW_DATA_DIR, DATASET_NAME
data_dir = os.path.join(RAW_DATA_DIR,DATASET_NAME)

print(data_dir)
data = read_data(data_dir)

/Users/amir/Desktop/GitHub/ClinicalAI-Toolkit/datasets/raw/Medical_data.json


In [18]:
data.shape

(99999, 6)

In [19]:
data.isna().sum()

id               0
patient_info     0
clinical_note    0
annotations      0
questions        0
summary          0
dtype: int64

In [20]:
labels_col = pd.json_normalize(data['annotations'])

In [8]:
labels_col = pd.json_normalize(data['annotations'])
print(labels_col['classification'].unique().tolist())
print(labels_col['sentiment'].unique().tolist())
print(labels_col['classification'].value_counts())
print(labels_col['sentiment'].value_counts())
np.sum(labels_col.isna())
labels_col = labels_col.reset_index(drop=True)

data['clasification'] = labels_col['classification']

['low-risk', 'high-risk', 'moderate-risk']
['neutral', 'negative', 'positive']
classification
high-risk        33388
moderate-risk    33322
low-risk         33289
Name: count, dtype: int64
sentiment
negative    33431
neutral     33296
positive    33272
Name: count, dtype: int64


/Users/amir/Desktop/GitHub/ClinicalAI-Toolkit/venv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [22]:
data.head()
# data[labels_col.columns()]
data[labels_col.columns] = labels_col

In [23]:

data.drop(columns=['annotations'],inplace=True)


In [24]:
data.head()

,id,patient_info,clinical_note,questions,summary,classification,sentiment
0,record_10002,"{'age': '88', 'gender': 'non-binary', 'visit_d...",{'chief_complaint': 'Patient reports chest pai...,"[{'answer': 'Patient reports chest pain.', 'qu...",A 88-year-old non-binary presenting with Patie...,low-risk,neutral
1,record_10003,"{'age': '42', 'gender': 'female', 'visit_date'...",{'chief_complaint': 'Patient reports headache....,"[{'answer': 'Patient reports headache.', 'ques...",A 42-year-old female presenting with Patient r...,high-risk,negative
2,record_10004,"{'age': '8', 'gender': 'non-binary', 'visit_da...",{'chief_complaint': 'Patient reports shortness...,[{'answer': 'Patient reports shortness of brea...,A 8-year-old non-binary presenting with Patien...,moderate-risk,positive
3,record_10005,"{'age': '23', 'gender': 'male', 'visit_date': ...","{'chief_complaint': 'Patient reports nausea.',...","[{'answer': 'Patient reports nausea.', 'questi...",A 23-year-old male presenting with Patient rep...,low-risk,positive
4,record_10006,"{'age': '24', 'gender': 'male', 'visit_date': ...",{'chief_complaint': 'Patient reports shortness...,[{'answer': 'Patient reports shortness of brea...,A 24-year-old male presenting with Patient rep...,low-risk,neutral


In [26]:
print((data['sentiment'].value_counts())/data.shape[0])
print((data['classification'].value_counts()/data.shape[0]))

sentiment
negative    0.334313
neutral     0.332963
positive    0.332723
Name: count, dtype: float64
classification
high-risk        0.333883
moderate-risk    0.333223
low-risk         0.332893
Name: count, dtype: float64


In [45]:
data['id'][0]

'record_000001'

In [44]:
data['metadata'][0]

{'age': 67,
 'gender': 'male',
 'visit_type': 'inpatient',
 'visit_date': '2025-01-02T18:29:31'}

In [46]:
data['questions'][0]

[{'question': "What are the patient's symptoms?",
  'answer': 'shortness of breath'},
 {'question': 'What is the diagnosis?', 'answer': 'pneumonia'},
 {'question': 'What is the treatment plan?',
  'answer': 'albuterol nebulization, aspirin'}]

In [27]:
data['sentiment'][0]

'neutral'

In [30]:
print(data['summary'][0])

A 88-year-old non-binary presenting with Patient reports chest pain.. Diagnosis: COPD. Plan: Administer aspirin and nitroglycerin..


In [29]:
data['summary'][49]

'A 85-year-old male presenting with Patient reports fatigue.. Diagnosis: migraine. Plan: Start inhaled corticosteroids and bronchodilators..'